### Diferencia entre usar un modelo y entrenarlo
**Usamos** un modelo cuando le proporcionamos entradas y recibimos una estimación o predicción, lo hacemos cuando lo entrenamos o lo usamos en el mundo real, usarlo solo nos lleva pocos segundos.

**Entrenamos** un modelo para mejorar su funcionamiento, para ello se requiere de la función objetivo, el optimizado y el propio modelo en un bucle específico. Para entrenarlo nos puede llevar minutos o hasta días.

### Datos para usar, datos para entrenar
Dado un conjunto de datos puede que este tenga muchas columnas pero solo nos interesará las columnas que nuestro modelo acepte como entrada, **estas columnas se denominan características**.

Durante el entrenamiento, la función necesita saber la dalida del modelo como la respuesta correcta, a esto se le denomina **etiquetas**.

Entonces, para usar un modelo solo vamos a necesitar las **características**, mientras que durante el entrenamiento necesitaremos ambas, **características** y **etiquetas**.

No es necesario que cada vez que vaya usar el modelo tenga que entrenarlo, entonces usted puede guardarlo en disco y usarlo cuantas veces quiera, para lograr este cometido, el siguiente bloque de código nos ayudará con ello.

In [2]:
import pandas
import statsmodels.formula.api as smf
import joblib

# Load a file containing dog's boot and harness sizes
data = pandas.read_csv('doggy-boot-harness.csv')

# Print the first few rows
data.head()

# Create and train model
# Fit a simple model that finds a linear relationship
# between boot size and harness size, which we can use later
# to predict a dog's boot size, given their harness size
model = smf.ols(formula = "boot_size ~ harness_size", data = data).fit()

print("Model trained!")

# Save our model
# Our model is ready to use, but we don't need it yet. Let's save it to disk.
model_filename = './avalanche_dog_boot_model.pkl'
joblib.dump(model, model_filename)

print("Model saved!")

# Load our model (just for test)
model_loaded = joblib.load(model_filename)

print("We have loaded a model with the following parameters:")
print(model_loaded.params)

Model trained!
Model saved!
We have loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64


Ahora vamos a crear una función en la que podamos cargar nuestro modelo y poder usarlo posteriormente permitiendo pasar a la función la **característica** para poder obtener una predicción, en este escenario el tamaño de la bota de un lomito según el tamaño de su arnés.

In [3]:
# Let's write a function that loads and uses our model
def load_model_and_predict(harness_size):
    '''
    This function loads a pretrained model. It uses the model
    with the customer's dog's harness size to predict the size of
    boots that will fit that dog.

    harness_size: The dog harness size, in cm 
    '''

    # Load the model from file and print basic information about it
    loaded_model = joblib.load(model_filename)

    print("We've loaded a model with the following parameters:")
    print(loaded_model.params)

    # Prepare data for the model
    inputs = {"harness_size":[harness_size]} 

    # Use the model to make a prediction
    predicted_boot_size = loaded_model.predict(inputs)[0]

    return predicted_boot_size

# Practice using our model
predicted_boot_size = load_model_and_predict(45)

print("Predicted dog boot size:", predicted_boot_size)

We've loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64
Predicted dog boot size: 32.085753565904774


### Usano nuestro modelo en el mundo real
Como ejemplo, haremos una función que acepte el tamaño del arnés, el tamaño de las botas seleccionadas y devuelva un mensaje para el cliente. Integraríamos esta función en nuestra tienda online.

In [4]:
def check_size_of_boots(selected_harness_size, selected_boot_size):
    '''
    Calculates whether the customer has chosen a pair of doggy boots that 
    are a sensible size. This works by estimating the dog's actual boot 
    size from their harness size.

    This returns a message for the customer that should be shown before
    they complete their payment 

    selected_harness_size: The size of the harness the customer wants to buy
    selected_boot_size: The size of the doggy boots the customer wants to buy
    '''

    # Estimate the customer's dog's boot size
    estimated_boot_size = load_model_and_predict(selected_harness_size)

    # Round to the nearest whole number because we don't sell partial sizes
    estimated_boot_size = int(round(estimated_boot_size))

    # Check if the boot size selected is appropriate
    if selected_boot_size == estimated_boot_size:
        # The selected boots are probably OK
        return f"Great choice! We think these boots will fit your avalanche dog well."

    if selected_boot_size < estimated_boot_size:
        # Selected boots might be too small 
        return "The boots you have selected might be TOO SMALL for a dog as "\
               f"big as yours. We recommend a doggy boots size of {estimated_boot_size}."

    if selected_boot_size > estimated_boot_size:
        # Selected boots might be too big 
        return "The boots you have selected might be TOO BIG for a dog as "\
               f"small as yours. We recommend a doggy boots size of {estimated_boot_size}."
    

# Practice using our new warning system
check_size_of_boots(selected_harness_size=55, selected_boot_size=39)

We've loaded a model with the following parameters:
Intercept       5.719110
harness_size    0.585925
dtype: float64


'The boots you have selected might be TOO BIG for a dog as small as yours. We recommend a doggy boots size of 38.'